In [54]:
import numpy as np
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import yaml
from scipy.io import savemat, loadmat

In [55]:
def fix_mat(data):
    if data.dtype.names:
        new_data = dict()
        for name in data.dtype.names:
            new_data[name]=data[0][name][0]
        for k,v in new_data.items():
            if v.dtype.names:
                new_data[k] = fix_mat(v)
            else:
                new_data[k] = np.squeeze(v)
        return new_data
    else:
        return data

In [56]:
mat_file = 'features_extracted_4qdatest.mat'
data = loadmat(mat_file)

features = data['features_all']
trials_start = data['trials_start']

In [57]:
yaml_file = '/home/paolo/cvsa_ws/src/qda_cvsa/cfg/qda_h7.yaml'

with open(yaml_file, 'r') as file:
    params = yaml.safe_load(file)
    
# Create a new QDA model
qda = QuadraticDiscriminantAnalysis()

# Set the parameters
qda.priors_ = np.array(params['QdaCfg']['params']['priors'])
qda.means_ = np.array(params['QdaCfg']['params']['means'])
qda.rotations_ = np.array(params['QdaCfg']['params']['rotations'])
qda.scalings_ = np.array(params['QdaCfg']['params']['scalings'])
qda.covariance_ = np.array(params['QdaCfg']['params']['covs'])
qda.classes_ = np.array(params['QdaCfg']['params']['classlbs'])

# save parameters to extract the correct features
bands_features =  np.array(params['QdaCfg']['params']['band'])
idchans_features = np.array(params['QdaCfg']['params']['idchans']) - 1

In [58]:
# save the predictions of the qda
qda_prediction = []
for i in range(0, features.shape[0]):
    c_pred = qda.predict_proba(features[i].reshape(1, -1))
    qda_prediction.append(c_pred[0])
    
savemat('qda_prediction.mat', {'qda_prediction': qda_prediction})

In [65]:
# simulate the integrator
bufferSize = 80

final_4trial = []

for i in range(0, len(trials_start)-1):
    buffer = np.tile([730, 731], int(bufferSize/2))
    idx_buffer = 0
    for j in range(int(trials_start[i]), int(trials_start[i+1])):
        c_pred = qda_prediction[j]
        if c_pred[0] > c_pred[1]:
            buffer[idx_buffer] = 730
        else:
            buffer[idx_buffer] = 731
            
        if idx_buffer < bufferSize-1:
            idx_buffer += 1
        else:
            idx_buffer = 0
    
    final_4trial.append(np.array([np.sum(buffer==730)/bufferSize, np.sum(buffer==731)/bufferSize]))
    
savemat('final_4trial.mat', {'final_4trial': final_4trial})